In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('C:/develop/Fishing_team/data/dataset.csv')
df['datetime'] = pd.to_datetime(df['datetime'])

In [3]:
df['card_status'].value_counts()

card_status
active     27451
act        25539
blk           48
blocked       41
Name: count, dtype: int64

In [ ]:
df = df.drop(columns=['Unnamed: 0', 'transaction_id', 'ip', 'mcc', 'card_type', 'expiration_date'])

In [5]:
df['time_diff'] = df.groupby('client_id')['datetime'].diff().fillna(pd.Timedelta(0))
df['time_diff'] = df['time_diff'].dt.total_seconds()

In [6]:
df['card_status'] = df['card_status'].map({'active': 0, 'act': 0, 'blocked': 1, 'blk': 1})

In [7]:
categorical_columns = ['device_type', 'tran_code', 'oper_type', 'card_status']
le = LabelEncoder()
for col in categorical_columns:
    df[col] = le.fit_transform(df[col])

In [8]:
numeric_columns = ['device_id', 'pin_inc_count', 'sum', 'balance']
scaler = StandardScaler()
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

In [9]:
df = df.sort_values(by=['client_id', 'datetime'])

In [10]:
print(df.head())

      device_id  device_type  tran_code  client_id  pin_inc_count  \
6351  -0.886258            2          1  101648380      -0.364263   
6352  -0.843944            0          5  101648380      -0.364263   
6353  -1.358762            6          5  101648380      -0.364263   
6354   1.579697            2          1  101648380      -0.364263   
6355   1.081334            4          2  101648380      -0.364263   

      card_status            datetime       sum  oper_type   balance  \
6351            0 2022-02-09 17:50:08 -0.466816         11  1.023095   
6352            0 2022-02-21 19:39:05 -0.467969         13  1.021239   
6353            0 2022-03-15 04:35:03 -0.468374         13  1.019384   
6354            0 2022-03-26 19:32:45 -0.467283         10  1.021241   
6355            0 2022-04-16 02:54:10 -0.468280          5  1.019385   

      time_diff  
6351        0.0  
6352  1043337.0  
6353  1846558.0  
6354  1004262.0  
6355  1754485.0  


In [11]:
sequence_length = 10

In [12]:
def create_sequences(df, sequence_length):
    sequences = []
    labels = []

    for client_id, group in df.groupby('client_id'):
        for i in range(len(group) - sequence_length):
            sequence = group.iloc[i:i+sequence_length][numeric_columns + categorical_columns + ['time_diff']].values
            label = group.iloc[i + sequence_length]['sum']  # Используем сумму как метку для аномалии (можно адаптировать под другие задачи)
            sequences.append(sequence)
            labels.append(label)
    
    return np.array(sequences), np.array(labels)

In [13]:
X, y = create_sequences(df, sequence_length)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = Sequential()

In [ ]:
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True,
               kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Добавляем второй LSTM слой
model.add(LSTM(64, activation='relu', return_sequences=True, kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Добавляем Dense слой
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))

# Добавляем выходной слой
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse', metrics=['mse'])


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('C:/develop/Fishing_team/data/dataset.csv')
df['datetime'] = pd.to_datetime(df['datetime'])

In [ ]:
df['card_status'].value_counts()

card_status
active     27451
act        25539
blk           48
blocked       41
Name: count, dtype: int64

In [ ]:
df = df.drop(columns=['Unnamed: 0', 'transaction_id', 'ip', 'mcc', 'card_type', 'expiration_date'])

In [ ]:
df['time_diff'] = df.groupby('client_id')['datetime'].diff().fillna(pd.Timedelta(0))
df['time_diff'] = df['time_diff'].dt.total_seconds()

In [ ]:
df['card_status'] = df['card_status'].map({'active': 0, 'act': 0, 'blocked': 1, 'blk': 1})

In [ ]:
categorical_columns = ['device_type', 'tran_code', 'oper_type', 'card_status']
le = LabelEncoder()
for col in categorical_columns:
    df[col] = le.fit_transform(df[col])

In [ ]:
numeric_columns = ['device_id', 'pin_inc_count', 'sum', 'balance']
scaler = StandardScaler()
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

In [ ]:
df = df.sort_values(by=['client_id', 'datetime'])

In [ ]:
print(df.head())

      device_id  device_type  tran_code  client_id  pin_inc_count  \
6351  -0.886258            2          1  101648380      -0.364263   
6352  -0.843944            0          5  101648380      -0.364263   
6353  -1.358762            6          5  101648380      -0.364263   
6354   1.579697            2          1  101648380      -0.364263   
6355   1.081334            4          2  101648380      -0.364263   

      card_status            datetime       sum  oper_type   balance  \
6351            0 2022-02-09 17:50:08 -0.466816         11  1.023095   
6352            0 2022-02-21 19:39:05 -0.467969         13  1.021239   
6353            0 2022-03-15 04:35:03 -0.468374         13  1.019384   
6354            0 2022-03-26 19:32:45 -0.467283         10  1.021241   
6355            0 2022-04-16 02:54:10 -0.468280          5  1.019385   

      time_diff  
6351        0.0  
6352  1043337.0  
6353  1846558.0  
6354  1004262.0  
6355  1754485.0  


In [ ]:
sequence_length = 10

In [ ]:
def create_sequences(df, sequence_length):
    sequences = []
    labels = []

    for client_id, group in df.groupby('client_id'):
        for i in range(len(group) - sequence_length):
            sequence = group.iloc[i:i+sequence_length][numeric_columns + categorical_columns + ['time_diff']].values
            label = group.iloc[i + sequence_length]['sum']  # Используем сумму как метку для аномалии (можно адаптировать под другие задачи)
            sequences.append(sequence)
            labels.append(label)
    
    return np.array(sequences), np.array(labels)

In [ ]:
X, y = create_sequences(df, sequence_length)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = Sequential()

In [ ]:
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True,
               kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Добавляем второй LSTM слой
model.add(LSTM(64, activation='relu', return_sequences=True, kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Добавляем Dense слой
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))

# Добавляем выходной слой
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse', metrics=['mse'])


In [ ]:
import optuna
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
import tensorflow as tf

# Функция для проверки устройства, которое используется
def get_device_info():
    physical_devices = tf.config.list_physical_devices()
    for device in physical_devices:
        print(f"Device: {device}, Type: {device.device_type}")
    # Если доступен GPU, его информация будет выведена
    if tf.config.list_physical_devices('GPU'):
        print("GPU is available and will be used for training.")
    else:
        print("GPU is not available. CPU will be used for training.")

# Выводим информацию о доступных устройствах (GPU или CPU)
get_device_info()

# Настроим Optuna для логирования
optuna.logging.set_verbosity(optuna.logging.INFO)

def create_model(trial):
    # Примерный процесс создания модели с использованием trial (кандидатных гиперпараметров)
    model = Sequential()

    # Выбор количества нейронов в первом LSTM слое
    lstm_units_1 = trial.suggest_int('lstm_units_1', 32, 128, step=32)
    print(f"Trying lstm_units_1 = {lstm_units_1}")
    model.add(LSTM(lstm_units_1, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True,
                   kernel_regularizer=regularizers.l2(0.01)))
    model.add(BatchNormalization())
    dropout_1 = trial.suggest_float('dropout', 0.2, 0.5)
    print(f"Trying dropout = {dropout_1}")
    model.add(Dropout(dropout_1))

    # Выбор количества нейронов во втором LSTM слое
    lstm_units_2 = trial.suggest_int('lstm_units_2', 32, 128, step=32)
    print(f"Trying lstm_units_2 = {lstm_units_2}")
    model.add(LSTM(lstm_units_2, activation='relu', return_sequences=False, kernel_regularizer=regularizers.l2(0.01)))
    model.add(BatchNormalization())
    dropout_2 = trial.suggest_float('dropout', 0.2, 0.5)
    print(f"Trying dropout = {dropout_2}")
    model.add(Dropout(dropout_2))

    # Добавляем Dense слой
    dense_dropout = trial.suggest_float('dropout_dense', 0.2, 0.5)
    print(f"Trying dense dropout = {dense_dropout}")
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dense_dropout))

    # Выходной слой
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mse', metrics=['mse'])
    return model

def objective(trial):
    # Создаем модель с гиперпараметрами, выбранными с помощью Optuna
    model = create_model(trial)
    
    # Используем EarlyStopping, чтобы избежать переобучения
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    
    # Обучаем модель
    print(f"Training trial {trial.number}")
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), 
                        callbacks=[early_stopping], verbose=0)
    
    # Оцениваем модель на тестовых данных
    test_loss, test_mse = model.evaluate(X_test, y_test, verbose=0)
    print(f"Trial {trial.number} - test_loss: {test_loss}, test_mse: {test_mse}")
    
    # Возвращаем потери, которые будут минимизировать Optuna
    return test_loss

# Создаем объект study для поиска гиперпараметров
study = optuna.create_study(direction='minimize')

# Запускаем поиск
study.optimize(objective, n_trials=50)  # пробуем 50 разных комбинаций гиперпараметров

# Выводим лучшие гиперпараметры
print(f"Best hyperparameters: {study.best_params}")

# Выводим детали о лучшем trial
best_trial = study.best_trial
print(f"Best trial: {best_trial.number}")
print(f"Best trial value: {best_trial.value}")
print(f"Best trial parameters: {best_trial.params}")


[I 2024-11-30 03:47:22,536] A new study created in memory with name: no-name-163ee4eb-bf22-410f-b5b6-7f4095a3809c


Device: PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), Type: CPU
GPU is not available. CPU will be used for training.
Trying lstm_units_1 = 32
Trying dropout = 0.4895099386054773
Trying lstm_units_2 = 32
Trying dropout = 0.4895099386054773
Trying dense dropout = 0.4858500330765257
Training trial 0


[I 2024-11-30 03:50:43,985] Trial 0 finished with value: 0.9949425458908081 and parameters: {'lstm_units_1': 32, 'dropout': 0.4895099386054773, 'lstm_units_2': 32, 'dropout_dense': 0.4858500330765257}. Best is trial 0 with value: 0.9949425458908081.


Trial 0 - test_loss: 0.9949425458908081, test_mse: 0.9946932792663574
Trying lstm_units_1 = 32
Trying dropout = 0.3169754485230837
Trying lstm_units_2 = 128
Trying dropout = 0.3169754485230837
Trying dense dropout = 0.49196023881439305
Training trial 1


[I 2024-11-30 03:56:53,455] Trial 1 finished with value: 0.9946401715278625 and parameters: {'lstm_units_1': 32, 'dropout': 0.3169754485230837, 'lstm_units_2': 128, 'dropout_dense': 0.49196023881439305}. Best is trial 1 with value: 0.9946401715278625.


...

Trial 47 - test_loss: 0.9961532950401306, test_mse: 0.9946945905685425
Trying lstm_units_1 = 96
Trying dropout = 0.33059835000058185
Trying lstm_units_2 = 128
Trying dropout = 0.33059835000058185
Trying dense dropout = 0.4612423106393342
Training trial 48


[I 2024-11-30 07:13:19,468] Trial 48 finished with value: 0.9946451783180237 and parameters: {'lstm_units_1': 96, 'dropout': 0.33059835000058185, 'lstm_units_2': 128, 'dropout_dense': 0.4612423106393342}. Best is trial 4 with value: 0.9837175607681274.


Trial 48 - test_loss: 0.9946451783180237, test_mse: 0.9946373105049133
Trying lstm_units_1 = 96
Trying dropout = 0.33716174554726885
Trying lstm_units_2 = 96
Trying dropout = 0.33716174554726885
Trying dense dropout = 0.41396273669382105
Training trial 49


[I 2024-11-30 07:19:23,660] Trial 49 finished with value: 0.9953534603118896 and parameters: {'lstm_units_1': 96, 'dropout': 0.33716174554726885, 'lstm_units_2': 96, 'dropout_dense': 0.41396273669382105}. Best is trial 4 with value: 0.9837175607681274.


Trial 49 - test_loss: 0.9953534603118896, test_mse: 0.9946703910827637
Best hyperparameters: {'lstm_units_1': 32, 'dropout': 0.3116036152445094, 'lstm_units_2': 128, 'dropout_dense': 0.44274738946248593}
Best trial: 4
Best trial value: 0.9837175607681274
Best trial parameters: {'lstm_units_1': 32, 'dropout': 0.3116036152445094, 'lstm_units_2': 128, 'dropout_dense': 0.44274738946248593}


In [ ]:
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

In [ ]:
test_loss, test_mse = model.evaluate(X_test, y_test)
print(f'Loss on test set: {test_loss}, MSE: {test_mse}')

In [ ]:
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
plt.plot(history.history['mse'], label='train_mse')
plt.plot(history.history['val_mse'], label='val_mse')
plt.legend()
plt.title('Training and Validation MSE')
plt.show()

In [16]:
import optuna
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
import tensorflow as tf

# Функция для проверки устройства, которое используется
def get_device_info():
    physical_devices = tf.config.list_physical_devices()
    for device in physical_devices:
        print(f"Device: {device}, Type: {device.device_type}")
    # Если доступен GPU, его информация будет выведена
    if tf.config.list_physical_devices('GPU'):
        print("GPU is available and will be used for training.")
    else:
        print("GPU is not available. CPU will be used for training.")

# Выводим информацию о доступных устройствах (GPU или CPU)
get_device_info()

# Настроим Optuna для логирования
optuna.logging.set_verbosity(optuna.logging.INFO)

def create_model(trial):
    # Примерный процесс создания модели с использованием trial (кандидатных гиперпараметров)
    model = Sequential()

    # Выбор количества нейронов в первом LSTM слое
    lstm_units_1 = trial.suggest_int('lstm_units_1', 32, 128, step=32)
    print(f"Trying lstm_units_1 = {lstm_units_1}")
    model.add(LSTM(lstm_units_1, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True,
                   kernel_regularizer=regularizers.l2(0.01)))
    model.add(BatchNormalization())
    dropout_1 = trial.suggest_float('dropout', 0.2, 0.5)
    print(f"Trying dropout = {dropout_1}")
    model.add(Dropout(dropout_1))

    # Выбор количества нейронов во втором LSTM слое
    lstm_units_2 = trial.suggest_int('lstm_units_2', 32, 128, step=32)
    print(f"Trying lstm_units_2 = {lstm_units_2}")
    model.add(LSTM(lstm_units_2, activation='relu', return_sequences=False, kernel_regularizer=regularizers.l2(0.01)))
    model.add(BatchNormalization())
    dropout_2 = trial.suggest_float('dropout', 0.2, 0.5)
    print(f"Trying dropout = {dropout_2}")
    model.add(Dropout(dropout_2))

    # Добавляем Dense слой
    dense_dropout = trial.suggest_float('dropout_dense', 0.2, 0.5)
    print(f"Trying dense dropout = {dense_dropout}")
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dense_dropout))

    # Выходной слой
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mse', metrics=['mse'])
    return model

def objective(trial):
    # Создаем модель с гиперпараметрами, выбранными с помощью Optuna
    model = create_model(trial)
    
    # Используем EarlyStopping, чтобы избежать переобучения
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    
    # Обучаем модель
    print(f"Training trial {trial.number}")
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), 
                        callbacks=[early_stopping], verbose=0)
    
    # Оцениваем модель на тестовых данных
    test_loss, test_mse = model.evaluate(X_test, y_test, verbose=0)
    print(f"Trial {trial.number} - test_loss: {test_loss}, test_mse: {test_mse}")
    
    # Возвращаем потери, которые будут минимизировать Optuna
    return test_loss

# Создаем объект study для поиска гиперпараметров
study = optuna.create_study(direction='minimize')

# Запускаем поиск
study.optimize(objective, n_trials=50)  # пробуем 50 разных комбинаций гиперпараметров

# Выводим лучшие гиперпараметры
print(f"Best hyperparameters: {study.best_params}")

# Выводим детали о лучшем trial
best_trial = study.best_trial
print(f"Best trial: {best_trial.number}")
print(f"Best trial value: {best_trial.value}")
print(f"Best trial parameters: {best_trial.params}")


[I 2024-11-30 03:47:22,536] A new study created in memory with name: no-name-163ee4eb-bf22-410f-b5b6-7f4095a3809c


Device: PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), Type: CPU
GPU is not available. CPU will be used for training.
Trying lstm_units_1 = 32
Trying dropout = 0.4895099386054773
Trying lstm_units_2 = 32
Trying dropout = 0.4895099386054773
Trying dense dropout = 0.4858500330765257
Training trial 0


[I 2024-11-30 03:50:43,985] Trial 0 finished with value: 0.9949425458908081 and parameters: {'lstm_units_1': 32, 'dropout': 0.4895099386054773, 'lstm_units_2': 32, 'dropout_dense': 0.4858500330765257}. Best is trial 0 with value: 0.9949425458908081.


Trial 0 - test_loss: 0.9949425458908081, test_mse: 0.9946932792663574
Trying lstm_units_1 = 32
Trying dropout = 0.3169754485230837
Trying lstm_units_2 = 128
Trying dropout = 0.3169754485230837
Trying dense dropout = 0.49196023881439305
Training trial 1


[I 2024-11-30 03:56:53,455] Trial 1 finished with value: 0.9946401715278625 and parameters: {'lstm_units_1': 32, 'dropout': 0.3169754485230837, 'lstm_units_2': 128, 'dropout_dense': 0.49196023881439305}. Best is trial 1 with value: 0.9946401715278625.


...

Trial 47 - test_loss: 0.9961532950401306, test_mse: 0.9946945905685425
Trying lstm_units_1 = 96
Trying dropout = 0.33059835000058185
Trying lstm_units_2 = 128
Trying dropout = 0.33059835000058185
Trying dense dropout = 0.4612423106393342
Training trial 48


[I 2024-11-30 07:13:19,468] Trial 48 finished with value: 0.9946451783180237 and parameters: {'lstm_units_1': 96, 'dropout': 0.33059835000058185, 'lstm_units_2': 128, 'dropout_dense': 0.4612423106393342}. Best is trial 4 with value: 0.9837175607681274.


Trial 48 - test_loss: 0.9946451783180237, test_mse: 0.9946373105049133
Trying lstm_units_1 = 96
Trying dropout = 0.33716174554726885
Trying lstm_units_2 = 96
Trying dropout = 0.33716174554726885
Trying dense dropout = 0.41396273669382105
Training trial 49


[I 2024-11-30 07:19:23,660] Trial 49 finished with value: 0.9953534603118896 and parameters: {'lstm_units_1': 96, 'dropout': 0.33716174554726885, 'lstm_units_2': 96, 'dropout_dense': 0.41396273669382105}. Best is trial 4 with value: 0.9837175607681274.


Trial 49 - test_loss: 0.9953534603118896, test_mse: 0.9946703910827637
Best hyperparameters: {'lstm_units_1': 32, 'dropout': 0.3116036152445094, 'lstm_units_2': 128, 'dropout_dense': 0.44274738946248593}
Best trial: 4
Best trial value: 0.9837175607681274
Best trial parameters: {'lstm_units_1': 32, 'dropout': 0.3116036152445094, 'lstm_units_2': 128, 'dropout_dense': 0.44274738946248593}


In [ ]:
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

In [ ]:
test_loss, test_mse = model.evaluate(X_test, y_test)
print(f'Loss on test set: {test_loss}, MSE: {test_mse}')

In [ ]:
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
plt.plot(history.history['mse'], label='train_mse')
plt.plot(history.history['val_mse'], label='val_mse')
plt.legend()
plt.title('Training and Validation MSE')
plt.show()